In [ ]:
from __future__ import annotations

import os
from pathlib import Path
from datetime import datetime

ROOT = Path.cwd().resolve()
if ROOT.name == "notebooks_v2":
    os.chdir(ROOT.parent)
ROOT = Path.cwd().resolve()
assert (ROOT / "src").exists(), f"Not at repo root. CWD={ROOT}"
print("✅ CWD =", ROOT)

def backup_write(rel_path: str, content: str) -> None:
    p = ROOT / rel_path
    p.parent.mkdir(parents=True, exist_ok=True)
    if p.exists():
        ts = datetime.now().strftime("%Y%m%d_%H%M%S")
        bak = p.with_suffix(p.suffix + f".bak_{ts}")
        bak.write_text(p.read_text(encoding="utf-8"), encoding="utf-8")
        print("BACKUP:", bak)
    p.write_text(content, encoding="utf-8")
    print("WROTE:", p)

# -------------------------------
# 1) Backend: run_manager_service.py
#    - store selected_job_urls
#    - support reject_ranking / reject_drafts
# -------------------------------
backup_write("src/careeragent/api/run_manager_service.py", r'''
from __future__ import annotations

import asyncio
import concurrent.futures as cf
import copy
import threading
from datetime import datetime, timezone
from pathlib import Path
from typing import Any, Dict, Optional
from uuid import uuid4

from careeragent.services.db_service import SqliteStateStore
from careeragent.langgraph.runtime_nodes import run_single_layer, approve_ranking_flow, approve_drafts_flow


def utc_now() -> str:
    return datetime.now(timezone.utc).strftime("%Y-%m-%dT%H:%M:%SZ")


def artifacts_root() -> Path:
    return Path("src/careeragent/artifacts").resolve()


class RunManagerService:
    """
    Description: Indestructible background runner with hard timeouts + HITL actions.
    Layer: L8
    """

    def __init__(self) -> None:
        self._store = SqliteStateStore()

    def _runs_dir(self, run_id: str) -> Path:
        d = artifacts_root() / "runs" / run_id
        d.mkdir(parents=True, exist_ok=True)
        return d

    def save_state(self, *, run_id: str, state: Dict[str, Any]) -> None:
        state.setdefault("meta", {})
        state["meta"]["heartbeat_utc"] = utc_now()
        self._store.upsert_state(run_id=run_id, status=str(state.get("status", "unknown")), state=state, updated_at_utc=utc_now())

    def get_state(self, run_id: str) -> Optional[Dict[str, Any]]:
        return self._store.get_state(run_id=run_id)

    def create_run(self, *, resume_filename: str, resume_text: str, resume_bytes: bytes, preferences: Dict[str, Any]) -> Dict[str, Any]:
        run_id = uuid4().hex
        run_dir = self._runs_dir(run_id)
        (run_dir / "resume_upload.bin").write_bytes(resume_bytes)
        (run_dir / "resume_raw.txt").write_text(resume_text, encoding="utf-8")

        thresholds = (preferences.get("thresholds") or {})
        if "default" in thresholds:
            d = float(thresholds["default"])
            thresholds.setdefault("parser", d)
            thresholds.setdefault("discovery", d)
            thresholds.setdefault("match", d)
            thresholds.setdefault("draft", d)

        state: Dict[str, Any] = {
            "run_id": run_id,
            "status": "running",
            "pending_action": None,
            "hitl_reason": None,
            "hitl_payload": {},
            "thresholds": thresholds,
            "max_retries": int(preferences.get("max_refinements", 3)),
            "layer_retry_count": {},
            "preferences": preferences,
            "resume_filename": resume_filename,
            "resume_text": resume_text,
            "profile": {},
            "jobs_raw": [],
            "jobs_scored": [],
            "ranking": [],
            "drafts": {},
            "bridge_docs": {},
            "meta": {
                "created_at_utc": utc_now(),
                "heartbeat_utc": utc_now(),
                "last_layer": None,
                "plan_layers": ["L0","L2","L3","L4","L5"],
                "approved_job_urls": [],
            },
            "steps": [],
            "live_feed": [{"layer":"L1","agent":"API","message":"Run created. Starting background pipeline…"}],
            "attempts": [],
            "gates": [],
            "evaluations": [],
            "artifacts": {
                "resume_raw": {"path": str(run_dir / "resume_raw.txt"), "content_type": "text/plain"},
                "resume_upload": {"path": str(run_dir / "resume_upload.bin"), "content_type": "application/octet-stream"},
            },
        }

        self.save_state(run_id=run_id, state=state)
        return state

    def start_background(self, run_id: str) -> None:
        t = threading.Thread(target=self._bg, args=(run_id,), daemon=True)
        t.start()

    def _call_layer(self, state: Dict[str, Any], layer: str) -> Dict[str, Any]:
        st_copy = copy.deepcopy(state)
        return asyncio.run(run_single_layer(st_copy, layer))

    def _bg(self, run_id: str) -> None:
        state = self.get_state(run_id)
        if not state:
            return

        plan = [("L0", 10), ("L2", 20), ("L3", 45), ("L4", 120), ("L5", 15)]

        for layer, tmo in plan:
            if state.get("status") in ("blocked","needs_human_approval","failed","completed"):
                self.save_state(run_id=run_id, state=state)
                return

            state.setdefault("meta", {})
            state["meta"]["last_layer"] = layer
            state.setdefault("steps", []).append({"layer_id": layer, "status": "running", "started_at_utc": utc_now()})
            state.setdefault("live_feed", []).append({"layer": layer, "agent":"Orchestrator", "message": f"Running {layer}…"})
            self.save_state(run_id=run_id, state=state)

            with cf.ThreadPoolExecutor(max_workers=1) as ex:
                fut = ex.submit(self._call_layer, state, layer)
                try:
                    state = fut.result(timeout=tmo)
                except cf.TimeoutError:
                    state["status"] = "needs_human_approval"
                    state["pending_action"] = f"timeout_{layer.lower()}"
                    state.setdefault("live_feed", []).append({"layer": layer, "agent":"TimeoutGuard", "message": f"{layer} timed out after {tmo}s"})
                except Exception as e:
                    state["status"] = "failed"
                    state["pending_action"] = f"error_{layer.lower()}"
                    state.setdefault("live_feed", []).append({"layer": layer, "agent":"CrashGuard", "message": f"{layer} crashed: {e}"})

            step_status = "ok"
            if str(state.get("pending_action","")).startswith(("timeout_","error_")):
                step_status = "failed"
            if state.get("status") in ("blocked","failed"):
                step_status = "failed"

            state["steps"][-1]["status"] = step_status
            state["steps"][-1]["finished_at_utc"] = utc_now()
            self.save_state(run_id=run_id, state=state)

            if state.get("pending_action") == "review_ranking":
                state["status"] = "needs_human_approval"
                self.save_state(run_id=run_id, state=state)
                return

        if state.get("status") == "running":
            state["status"] = "needs_human_approval"
            state["pending_action"] = "review_ranking"
            state.setdefault("live_feed", []).append({"layer":"L5","agent":"Orchestrator","message":"Ranking ready for review."})
            self.save_state(run_id=run_id, state=state)

    async def handle_action(self, *, run_id: str, action_type: str, payload: Dict[str, Any]) -> Dict[str, Any]:
        state = self.get_state(run_id)
        if not state:
            raise ValueError("run_id not found")

        state.setdefault("meta", {})
        state["meta"]["last_user_action"] = {"type": action_type, "payload": payload, "at_utc": utc_now()}

        if action_type == "execute_layer":
            layer = str(payload.get("layer","")).upper()
            state = await asyncio.to_thread(lambda: self._call_layer(state, layer))
            self.save_state(run_id=run_id, state=state)
            return state

        if action_type == "approve_ranking":
            selected = payload.get("selected_job_urls") or []
            if isinstance(selected, list):
                state["meta"]["approved_job_urls"] = [str(u) for u in selected if str(u).strip()]
            state.setdefault("live_feed", []).append({"layer":"L5","agent":"HITL","message": f"Ranking approved. Selected jobs={len(state['meta']['approved_job_urls'])}."})
            state = await asyncio.to_thread(lambda: asyncio.run(approve_ranking_flow(copy.deepcopy(state))))
            self.save_state(run_id=run_id, state=state)
            return state

        if action_type == "reject_ranking":
            reason = str(payload.get("reason","")).strip()
            state.setdefault("meta", {}).setdefault("ranking_reject_reasons", []).append(reason or "no_reason")
            state["status"] = "running"
            state["pending_action"] = None
            state.setdefault("live_feed", []).append({"layer":"L5","agent":"HITL","message": f"Ranking rejected. Re-running hunt. Reason: {reason[:140]}"} )
            self.save_state(run_id=run_id, state=state)
            self.start_background(run_id)
            return state

        if action_type == "approve_drafts":
            state.setdefault("live_feed", []).append({"layer":"L6","agent":"HITL","message":"Drafts approved. Finalizing…"})
            state = await asyncio.to_thread(lambda: asyncio.run(approve_drafts_flow(copy.deepcopy(state))))
            self.save_state(run_id=run_id, state=state)
            return state

        if action_type == "reject_drafts":
            reason = str(payload.get("reason","")).strip()
            state.setdefault("meta", {}).setdefault("draft_reject_reasons", []).append(reason or "no_reason")
            state["status"] = "needs_human_approval"
            state["pending_action"] = "review_ranking"
            state.setdefault("live_feed", []).append({"layer":"L6","agent":"HITL","message": f"Drafts rejected. Back to ranking. Reason: {reason[:140]}"} )
            self.save_state(run_id=run_id, state=state)
            return state

        state.setdefault("live_feed", []).append({"layer":"L5","agent":"HITL","message":f"Unhandled action_type={action_type}"})
        self.save_state(run_id=run_id, state=state)
        return state
''')

# -------------------------------
# 2) runtime_nodes.py: filter ranking by selected_job_urls + fallback draft writer
# -------------------------------
backup_write("src/careeragent/langgraph/runtime_nodes.py", r'''
from __future__ import annotations

import hashlib
import os
from pathlib import Path
from typing import Any, Dict, List

from careeragent.langgraph.runtime_nodes import run_single_layer as _run_single_layer_base  # type: ignore

# NOTE:
# You already have a working local-first runtime_nodes.py in your repo.
# We patch only the approval flows here by re-defining approve_* functions.
# This file will be imported by RunManagerService for approve_ranking_flow/approve_drafts_flow.

# If you already have approve_* in this module, this overwrite is intentional (backup created).

def _artifacts_root() -> Path:
    return Path("src/careeragent/artifacts").resolve()

def _runs_dir(run_id: str) -> Path:
    d = _artifacts_root() / "runs" / run_id
    d.mkdir(parents=True, exist_ok=True)
    return d

def _write_text(p: Path, txt: str) -> str:
    p.parent.mkdir(parents=True, exist_ok=True)
    p.write_text(txt, encoding="utf-8")
    return str(p)

def _job_key(url: str) -> str:
    return hashlib.md5(url.encode("utf-8"), usedforsecurity=False).hexdigest()[:12]  # noqa: S324

def _fallback_resume_md(profile: Dict[str, Any], job: Dict[str, Any]) -> str:
    name = (profile.get("name") or "Candidate").strip()
    contact = profile.get("contact") or {}
    skills = profile.get("skills") or []
    title = job.get("title") or "Target Role"
    matched = job.get("matched_skills") or []
    missing = job.get("missing_skills") or []

    return f"""# {name}
{contact.get('phone','')} | {contact.get('email','')}
{contact.get('linkedin','')}

## Target Role
**{title}**

## Professional Summary
AI/ML professional with strong delivery focus. Building production-ready GenAI + MLOps systems with measurable impact. Optimized for ATS scanning and recruiter readability.

## Core Skills (ATS)
{", ".join(skills[:25])}

## Matched Skills for This Role
{", ".join(matched[:20]) if matched else "See Skills section"}

## Skill Gaps (to close fast)
{", ".join(missing[:15]) if missing else "None detected from scrape text"}

## Experience
- Add 4–6 bullet points per role with metrics (latency, cost, accuracy, scale, adoption)
- Include tools + scope + outcomes per project

## Education
- MSIT (Healthcare Tech) — Clark University (in progress)

## Projects
- CareerAgent-AI: LangGraph agentic orchestration + HITL + explainability + analytics
"""

def _fallback_cover_md(profile: Dict[str, Any], job: Dict[str, Any], country: str) -> str:
    name = (profile.get("name") or "Candidate").strip()
    contact = profile.get("contact") or {}
    title = job.get("title") or "the role"
    company = job.get("company") or ""
    url = job.get("url") or ""

    greeting = "Dear Hiring Manager," if country.upper() == "US" else "Dear Hiring Team,"
    return f"""# Cover Letter — {name}

{contact.get('email','')} | {contact.get('phone','')} | {contact.get('linkedin','')}

{greeting}

I’m applying for **{title}** {("at " + company) if company else ""}. I build production-grade AI/ML and GenAI systems with strong MLOps discipline (tracking, reproducibility, governance). My experience aligns with high-impact delivery: building reliable pipelines, agentic workflows, and measurable model outcomes.

Highlights relevant to this role:
- End-to-end AI delivery: data → training → evaluation → deployment
- GenAI/LLM apps: RAG, orchestration, guardrails, prompt safety, evaluation loops
- MLOps + cloud: CI/CD, containers, monitoring, artifact/version control

I’d welcome the chance to discuss how I can help your team ship AI systems that are fast, safe, and measurable.

Sincerely,  
**{name}**

Job link: {url}
"""

async def run_single_layer(state: Dict[str, Any], layer: str) -> Dict[str, Any]:
    # delegate to your already-working local-first layer runner
    return await _run_single_layer_base(state, layer)

async def approve_ranking_flow(state: Dict[str, Any]) -> Dict[str, Any]:
    """
    Description: HITL approve ranking -> generate drafts for selected jobs.
    Layer: L6
    """
    run_id = str(state.get("run_id") or "run")
    prefs = state.get("preferences") or {}
    country = str(prefs.get("country") or "US")

    ranking: List[Dict[str, Any]] = state.get("ranking") or []
    approved = (state.get("meta") or {}).get("approved_job_urls") or []
    approved = [str(u) for u in approved if str(u).strip()]

    if approved:
        ranking = [j for j in ranking if str(j.get("url") or j.get("job_id") or "") in set(approved)]
    if not ranking:
        # fallback to top 10
        ranking = (state.get("ranking") or [])[:10]

    # Try your existing L6 node (if present)
    try:
        from careeragent.langgraph.nodes_l6_l9 import l6_draft_node, l6_evaluator_node
        state.update(await l6_draft_node(state))      # type: ignore[arg-type]
        state.update(await l6_evaluator_node(state))  # type: ignore[arg-type]
    except Exception:
        # Fallback deterministic draft generation
        prof = state.get("profile") or {}
        run_dir = _runs_dir(run_id)
        artifacts = state.setdefault("artifacts", {})
        drafts = []
        for job in ranking[:10]:
            url = str(job.get("url") or job.get("job_id") or "")
            key = _job_key(url or (job.get("title") or "job"))
            resume_p = run_dir / f"resume_{key}.md"
            cover_p = run_dir / f"cover_{key}.md"
            resume_txt = _fallback_resume_md(prof, job)
            cover_txt = _fallback_cover_md(prof, job, country)
            artifacts[f"resume_{key}"] = {"path": _write_text(resume_p, resume_txt), "content_type": "text/markdown"}
            artifacts[f"cover_{key}"] = {"path": _write_text(cover_p, cover_txt), "content_type": "text/markdown"}
            drafts.append({"job_url": url, "job_title": job.get("title"), "resume": artifacts[f"resume_{key}"]["path"], "cover": artifacts[f"cover_{key}"]["path"]})

        state["drafts"] = {"drafts": drafts}

    state["status"] = "needs_human_approval"
    state["pending_action"] = "review_drafts"
    state.setdefault("live_feed", []).append({"layer":"L6","agent":"HITL","message": f"Drafts generated for {min(10,len(ranking))} jobs. Review drafts."})
    return state

async def approve_drafts_flow(state: Dict[str, Any]) -> Dict[str, Any]:
    """
    Description: HITL approve drafts -> finalize.
    Layer: L7-L9
    """
    try:
        from careeragent.langgraph.nodes_l6_l9 import (
            l7_apply_node, l7_evaluator_node,
            l8_tracker_node, l8_evaluator_node,
            l9_analytics_node,
        )
        state.update(await l7_apply_node(state))      # type: ignore[arg-type]
        state.update(await l7_evaluator_node(state))  # type: ignore[arg-type]
        if state.get("status") == "needs_human_approval":
            return state
        state.update(await l8_tracker_node(state))      # type: ignore[arg-type]
        state.update(await l8_evaluator_node(state))    # type: ignore[arg-type]
        if state.get("status") == "needs_human_approval":
            return state
        state.update(await l9_analytics_node(state))    # type: ignore[arg-type]
    except Exception:
        # fallback: mark completed
        pass

    state["status"] = "completed"
    state["pending_action"] = None
    state.setdefault("live_feed", []).append({"layer":"L9","agent":"HITL","message":"Run completed."})
    return state
''')

# -------------------------------
# 3) Streamlit UI: show approval buttons + job grid + draft review
# -------------------------------
backup_write("app/ui/dashboard.py", r'''
from __future__ import annotations

import json
import os
from pathlib import Path
from typing import Any, Dict, List, Optional

import requests
import streamlit as st


def _api_base() -> str:
    return os.getenv("API_URL", "http://127.0.0.1:8000").rstrip("/")


def api_get(path: str) -> Dict[str, Any]:
    r = requests.get(_api_base() + path, timeout=30)
    r.raise_for_status()
    return r.json()


def api_post(path: str, payload: Dict[str, Any]) -> Dict[str, Any]:
    r = requests.post(_api_base() + path, json=payload, timeout=60)
    r.raise_for_status()
    return r.json()


def _load_artifact_json(artifact: Optional[Dict[str, Any]]) -> Optional[Any]:
    if not artifact:
        return None
    p = artifact.get("path")
    if not p:
        return None
    fp = Path(p)
    if not fp.exists():
        return None
    return json.loads(fp.read_text(encoding="utf-8"))


def _progress(state: Dict[str, Any]) -> int:
    steps = state.get("steps") or []
    done = sum(1 for s in steps if s.get("finished_at_utc"))
    total = max(1, len(steps) if steps else len((state.get("meta") or {}).get("plan_layers") or ["L0","L2","L3","L4","L5"]))
    return int((done / total) * 100)


def _job_card(job: Dict[str, Any]) -> None:
    url = job.get("url") or job.get("job_id") or ""
    title = job.get("title") or "(untitled)"
    mp = job.get("match_percent")
    comp = job.get("components") or {}
    st.markdown(f"**{title}**")
    cols = st.columns([1.2, 1.0, 1.0, 1.0, 1.3])
    cols[0].markdown(f"[Open job link]({url})")
    cols[1].metric("Match %", f"{mp:.1f}" if isinstance(mp, (int,float)) else str(mp))
    cols[2].metric("Skill", f"{(comp.get('skill_overlap',0)*100):.0f}%")
    cols[3].metric("Exp", f"{(comp.get('experience_alignment',0)*100):.0f}%")
    cols[4].metric("ATS", f"{(comp.get('ats_score',0)*100):.0f}%")
    if job.get("snippet"):
        st.caption(job["snippet"][:240])


def main() -> None:
    st.set_page_config(page_title="CareerAgent-AI — Glass-Box Mission Control", layout="wide")
    st.title("CareerAgent-AI — Glass-Box Mission Control")

    if "run_id" not in st.session_state:
        st.session_state["run_id"] = ""
    if "selected_urls" not in st.session_state:
        st.session_state["selected_urls"] = set()

    # Sidebar inputs
    with st.sidebar:
        st.markdown("### Preferences")
        default_th = st.slider("Default threshold", 0.0, 1.0, 0.70, 0.01)
        parser_th = st.slider("Parser threshold", 0.0, 1.0, 0.70, 0.01)
        discovery_th = st.slider("Discovery threshold", 0.0, 1.0, 0.70, 0.01)
        match_th = st.slider("Match threshold", 0.0, 1.0, 0.70, 0.01)
        max_ref = st.slider("Max retries / refinements", 0, 5, 3, 1)

        roles = st.text_input("Target roles (comma-separated)", "Data Scientist, ML Engineer, GenAI Engineer")
        location = st.text_input("Location", "United States")
        country = st.selectbox("Country", ["US", "CA", "UK", "IN", "AU"], index=0)
        remote = st.checkbox("Remote preferred", value=True)
        wfo_ok = st.checkbox("On-site/WFO acceptable", value=True)
        visa = st.checkbox("Visa sponsorship required (F1/OPT/H1B)", value=True)
        recency = st.number_input("Recency hours", min_value=6, max_value=168, value=36, step=6)
        max_jobs = st.number_input("Max jobs to hunt", min_value=10, max_value=60, value=40, step=5)
        phone = st.text_input("Phone for SMS (optional)", "")

        st.markdown("### Upload")
        up = st.file_uploader("Resume (PDF/TXT/DOCX)", type=["pdf","txt","docx"])

        if st.button("🚀 Start Hunt", use_container_width=True, disabled=(up is None)):
            prefs = {
                "target_roles": [r.strip() for r in roles.split(",") if r.strip()][:4],
                "country": country,
                "location": location,
                "remote": remote,
                "wfo_ok": wfo_ok,
                "salary": "",
                "visa_sponsorship_required": visa,
                "recency_hours": float(recency),
                "max_jobs": int(max_jobs),
                "max_refinements": int(max_ref),
                "user_phone": phone or None,
                "thresholds": {
                    "default": float(default_th),
                    "parser": float(parser_th),
                    "discovery": float(discovery_th),
                    "match": float(match_th),
                    "draft": float(default_th),
                },
            }
            files = {"resume": (up.name, up.getvalue(), up.type or "application/octet-stream")}
            data = {"preferences_json": json.dumps(prefs)}
            r = requests.post(_api_base() + "/analyze", files=files, data=data, timeout=120)
            r.raise_for_status()
            st.session_state["run_id"] = r.json()["run_id"]
            st.session_state["selected_urls"] = set()
            st.success(f"Run started: {st.session_state['run_id']}")

        st.markdown("---")
        st.markdown("### Existing Run")
        st.session_state["run_id"] = st.text_input("Run ID", st.session_state["run_id"])

    run_id = st.session_state["run_id"].strip()
    if not run_id:
        st.info("Upload a resume and click Start Hunt.")
        return

    # Pull status
    state = api_get(f"/status/{run_id}")

    # Header KPI row
    c1, c2, c3, c4 = st.columns([2,2,2,2])
    c1.markdown(f"**Run:** `{run_id}`")
    c2.markdown(f"**Status:** `{state.get('status')}`")
    c3.markdown(f"**Pending:** `{state.get('pending_action')}`")
    prog = _progress(state)
    c4.markdown(f"**Progress:** `{prog}%`")
    st.progress(prog / 100.0)

    # Live feed
    st.subheader("Live Agent Feed")
    feed = state.get("live_feed") or []
    for e in feed[-20:]:
        st.write(f"[{e.get('layer')} {e.get('agent')}] {e.get('message')}")

    tabs = st.tabs(["Pilot/Engineer", "Approval Grid", "Draft Review", "Learning Center", "Artifacts"])

    # -------- Pilot/Engineer
    with tabs[0]:
        st.markdown("### Engineer Controls")
        cols = st.columns(5)
        for i, layer in enumerate(["L0","L2","L3","L4","L5"]):
            if cols[i].button(f"Run {layer}", use_container_width=True):
                st = api_post(f"/action/{run_id}", {"action_type":"execute_layer", "payload":{"layer": layer}})
                st.success(f"Executed {layer}. Refresh.")
        st.caption("Use Approval Grid when pending_action=review_ranking.")

    # -------- Approval Grid (Ranking HITL)
    with tabs[1]:
        st.markdown("### Ranking Approval (HITL)")
        ranking = state.get("ranking") or []
        if not ranking:
            ranking = _load_artifact_json((state.get("artifacts") or {}).get("ranking")) or []

        if not ranking:
            st.warning("Ranking not available yet.")
        else:
            st.caption("Select jobs → Approve Selected to generate tailored drafts (resume + cover letter).")

            # default select top 10 if nothing selected yet
            if not st.session_state["selected_urls"]:
                for j in ranking[:10]:
                    u = j.get("url") or j.get("job_id") or ""
                    if u:
                        st.session_state["selected_urls"].add(u)

            # grid cards
            for idx, job in enumerate(ranking[:30], start=1):
                url = job.get("url") or job.get("job_id") or ""
                with st.expander(f"{idx}. {job.get('title','(untitled)')}", expanded=(idx<=3)):
                    left, right = st.columns([0.82, 0.18])
                    with left:
                        _job_card(job)
                    with right:
                        sel = st.checkbox("Select", value=(url in st.session_state["selected_urls"]), key=f"sel_{idx}")
                        if sel and url:
                            st.session_state["selected_urls"].add(url)
                        if (not sel) and url and url in st.session_state["selected_urls"]:
                            st.session_state["selected_urls"].remove(url)

            colA, colB = st.columns([1,1])
            with colA:
                if st.button("✅ Approve Selected → Generate Drafts", use_container_width=True, disabled=(len(st.session_state["selected_urls"])==0)):
                    api_post(f"/action/{run_id}", {"action_type":"approve_ranking", "payload":{"selected_job_urls": sorted(list(st.session_state["selected_urls"]))}})
                    st.success("Approved. Go to Draft Review tab and refresh.")
            with colB:
                reason = st.text_input("Reject reason (optional)", "")
                if st.button("❌ Reject Ranking → Re-run Hunt", use_container_width=True):
                    api_post(f"/action/{run_id}", {"action_type":"reject_ranking", "payload":{"reason": reason}})
                    st.warning("Rejected. Pipeline restarted.")

    # -------- Draft Review (after approve_ranking)
    with tabs[2]:
        st.markdown("### Draft Review (Resume + Cover Letter)")
        artifacts = state.get("artifacts") or {}
        # collect resume/cover artifacts
        resume_keys = [k for k in artifacts.keys() if k.startswith("resume_")]
        cover_keys = [k for k in artifacts.keys() if k.startswith("cover_")]

        if not resume_keys and not cover_keys:
            st.info("No drafts found yet. Approve ranking first.")
        else:
            st.caption("Preview and approve drafts to finalize (apply/tracker/analytics).")
            all_keys = sorted(list(set(resume_keys + cover_keys)))
            pick = st.selectbox("Select draft artifact", all_keys)
            if pick:
                p = artifacts[pick]["path"]
                txt = Path(p).read_text(encoding="utf-8") if Path(p).exists() else "(file missing)"
                st.code(txt[:8000], language="markdown")

                st.download_button("Download this file", data=txt.encode("utf-8"), file_name=Path(p).name, mime="text/markdown")

            col1, col2 = st.columns([1,1])
            with col1:
                if st.button("✅ Approve Drafts → Finalize (L7–L9)", use_container_width=True):
                    api_post(f"/action/{run_id}", {"action_type":"approve_drafts", "payload":{}})
                    st.success("Approved drafts. Refresh to see completed status.")
            with col2:
                r = st.text_input("Draft reject reason", "")
                if st.button("❌ Reject Drafts → Back to Ranking", use_container_width=True):
                    api_post(f"/action/{run_id}", {"action_type":"reject_drafts", "payload":{"reason": r}})
                    st.warning("Drafts rejected. Back to ranking.")

    # -------- Learning Center
    with tabs[3]:
        st.markdown("### Learning Center (Skill Gaps → Tutorials)")
        scored = state.get("jobs_scored") or []
        if not scored:
            scored = _load_artifact_json((state.get("artifacts") or {}).get("jobs_scored")) or []

        gaps = {}
        for j in scored[:30]:
            for sk in (j.get("missing_skills") or [])[:10]:
                gaps[sk] = gaps.get(sk, 0) + 1

        if not gaps:
            st.info("No skill gaps detected (or scrape text did not include explicit skills).")
        else:
            top = sorted(gaps.items(), key=lambda x: x[1], reverse=True)[:25]
            for sk, cnt in top:
                docs = f"https://www.google.com/search?q={sk}+official+documentation"
                yt = f"https://www.youtube.com/results?search_query={sk}+tutorial"
                st.write(f"**{sk}** (missing in {cnt} jobs) — [Docs]({docs}) | [YouTube]({yt})")

    # -------- Artifacts / Downloads
    with tabs[4]:
        st.markdown("### Artifacts (local-first)")
        artifacts = state.get("artifacts") or {}
        if not artifacts:
            st.info("No artifacts in state.")
        else:
            for k, v in artifacts.items():
                p = v.get("path","")
                st.write(f"- **{k}** → `{p}`")
        st.markdown("#### Quick downloads")
        for key in ("jobs_raw","jobs_scored","ranking","extracted_profile"):
            art = artifacts.get(key)
            if not art:
                continue
            fp = Path(art["path"])
            if fp.exists():
                st.download_button(f"Download {key}", data=fp.read_bytes(), file_name=fp.name, mime=art.get("content_type","application/octet-stream"))
''')

backup_write("app/main.py", r'''
from __future__ import annotations

from app.ui.dashboard import main

if __name__ == "__main__":
    main()
''')

print("✅ HITL UI + backend actions patched.")
print("Next: restart backend + restart Streamlit.")
''')